---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score

def blight_model():
    #######################################################################
    # Data Processing and Cleanup
    #######################################################################
    train = pd.read_csv('train.csv', encoding = "ISO-8859-1")
    test = pd.read_csv('test.csv', encoding = "ISO-8859-1")
    addresses = pd.read_csv('addresses.csv', encoding = "ISO-8859-1")
    latlons = pd.read_csv('latlons.csv', encoding = "ISO-8859-1")
    
    # Focus on the US
    train = train[train.country == 'USA']
    test = test[test.country == 'USA']
    
    # All tickets where the violators were found not responsible are not considered during evaluation
    train = train[np.isfinite(train['compliance'])]
    
    # Add latitude and longitude information to train/test sets

    train = pd.merge(train, pd.merge(addresses, latlons, on='address'), on='ticket_id')
    test = pd.merge(test, pd.merge(addresses, latlons, on='address'), on='ticket_id')
    
    # Columns that are not available in the test set
    columns_to_drop_train_set = ['payment_amount', 'balance_due', 'payment_date', 'payment_status', 'balance_due', 'collection_status', 'compliance_detail']
    
    # Fees are constant across all tickets
    columns_to_drop_all_sets = ['admin_fee', 'state_fee']
    
    # Drop redundant geolocation information. We only care about latitude and longitude
    columns_to_drop_all_sets = columns_to_drop_all_sets + ['violation_street_number', 'violation_street_name', 'violation_zip_code', 'mailing_address_str_number', 'mailing_address_str_name', 'city', 'state', 'zip_code', 'address', 'country', 'non_us_str_code']
    
    # Remove qualitative data such as names, descriptions, etc and information covered by other columns
    columns_to_drop_all_sets = columns_to_drop_all_sets + ['agency_name', 'inspector_name', 'violator_name', 'violation_description', 'grafitti_status']
    
    # Remove date related fields. We could use the time_gap between them but there is missing hearing_date data in the train set
    columns_to_drop_all_sets = columns_to_drop_all_sets + ['ticket_issued_date', 'hearing_date']
    
    # Removing redundant columns from train and test set
    train.drop(columns_to_drop_train_set + columns_to_drop_all_sets, axis=1, inplace=True)
    train_columns = list(train.columns.values)
    train_columns.remove('compliance')
    test = test[train_columns]
    
    # Making 'disposition' and 'violation_code' discrete values
    label_encoder = LabelEncoder()
    label_encoder.fit(train['disposition'].append(test['disposition'], ignore_index=True))
    train['disposition'] = label_encoder.transform(train['disposition'])
    test['disposition'] = label_encoder.transform(test['disposition'])
    
    label_encoder = LabelEncoder()
    label_encoder.fit(train['violation_code'].append(test['violation_code'], ignore_index=True))
    train['violation_code'] = label_encoder.transform(train['violation_code'])
    test['violation_code'] = label_encoder.transform(test['violation_code'])
    
    # Latitude and Longitude had some missing values. We cannot just remove them from the test set because a result is expected
    train = train.dropna()
    test.lat.fillna(method='pad', inplace=True)
    test.lon.fillna(method='pad', inplace=True)
    
    #######################################################################
    # Machine Learning Time
    #######################################################################
    X_train, X_test, y_train, y_test = train_test_split(train[train_columns], train['compliance'])
    
    regr_rf = RandomForestRegressor()
    grid_values = {'n_estimators': [10, 100], 'max_depth': [None, 30]}
    grid_clf_auc = GridSearchCV(regr_rf, param_grid=grid_values, scoring='roc_auc')
    grid_clf_auc.fit(X_train, y_train)
    
    return pd.DataFrame(grid_clf_auc.predict(test), test.ticket_id)

In [2]:
#######################################################################
# Data Processing and Cleanup
#######################################################################
train = pd.read_csv('train.csv', encoding = "ISO-8859-1")
test = pd.read_csv('test.csv', encoding = "ISO-8859-1")
addresses = pd.read_csv('addresses.csv', encoding = "ISO-8859-1")
latlons = pd.read_csv('latlons.csv', encoding = "ISO-8859-1")

# Focus on the US
train = train[train.country == 'USA']
test = test[test.country == 'USA']

# All tickets where the violators were found not responsible are not considered during evaluation
train = train[np.isfinite(train['compliance'])]

# Add latitude and longitude information to train/test sets

train = pd.merge(train, pd.merge(addresses, latlons, on='address'), on='ticket_id')
test = pd.merge(test, pd.merge(addresses, latlons, on='address'), on='ticket_id')

# Columns that are not available in the test set
columns_to_drop_train_set = ['payment_amount', 'balance_due', 'payment_date', 'payment_status', 'balance_due', 'collection_status', 'compliance_detail']

# Fees are constant across all tickets
columns_to_drop_all_sets = ['admin_fee', 'state_fee']

# Drop redundant geolocation information. We only care about latitude and longitude
columns_to_drop_all_sets = columns_to_drop_all_sets + ['violation_street_number', 'violation_street_name', 'violation_zip_code', 'mailing_address_str_number', 'mailing_address_str_name', 'city', 'state', 'zip_code', 'address', 'country', 'non_us_str_code']

# Remove qualitative data such as names, descriptions, etc and information covered by other columns
columns_to_drop_all_sets = columns_to_drop_all_sets + ['agency_name', 'inspector_name', 'violator_name', 'violation_description', 'grafitti_status']

# Remove date related fields. We could use the time_gap between them but there is missing hearing_date data in the train set
columns_to_drop_all_sets = columns_to_drop_all_sets + ['ticket_issued_date', 'hearing_date']

# Removing redundant columns from train and test set
train.drop(columns_to_drop_train_set + columns_to_drop_all_sets, axis=1, inplace=True)
train_columns = list(train.columns.values)
train_columns.remove('compliance')
test = test[train_columns]

# Making 'disposition' and 'violation_code' discrete values
label_encoder = LabelEncoder()
label_encoder.fit(train['disposition'].append(test['disposition'], ignore_index=True))
train['disposition'] = label_encoder.transform(train['disposition'])
test['disposition'] = label_encoder.transform(test['disposition'])

label_encoder = LabelEncoder()
label_encoder.fit(train['violation_code'].append(test['violation_code'], ignore_index=True))
train['violation_code'] = label_encoder.transform(train['violation_code'])
test['violation_code'] = label_encoder.transform(test['violation_code'])

# Latitude and Longitude had some missing values. We cannot just remove them from the test set because a result is expected
train = train.dropna()
test.lat.fillna(method='pad', inplace=True)
test.lon.fillna(method='pad', inplace=True)

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.columns

Index(['ticket_id', 'violation_code', 'disposition', 'fine_amount', 'late_fee',
       'discount_amount', 'clean_up_cost', 'judgment_amount', 'compliance',
       'lat', 'lon'],
      dtype='object')

In [4]:
train.head()

,ticket_id,violation_code,disposition,fine_amount,late_fee,discount_amount,clean_up_cost,judgment_amount,compliance,lat,lon
0,22056,175,5,250.0,25.0,0.0,0.0,305.0,0.0,42.390729,-83.124268
1,27586,98,6,750.0,75.0,0.0,0.0,855.0,1.0,42.326937,-83.135118
2,22046,175,5,250.0,25.0,0.0,0.0,305.0,0.0,42.145257,-83.208233
3,18738,97,5,750.0,75.0,0.0,0.0,855.0,0.0,42.433466,-83.023493
4,18735,96,5,100.0,10.0,0.0,0.0,140.0,0.0,42.388641,-83.037858


In [5]:
len(train)

159867

In [6]:
len(train.groupby('violation_code').count())

189

In [7]:
len(train.groupby('disposition').count())

4

In [8]:
train.isnull().sum(axis = 0)

ticket_id          0
violation_code     0
disposition        0
fine_amount        0
late_fee           0
discount_amount    0
clean_up_cost      0
judgment_amount    0
compliance         0
lat                0
lon                0
dtype: int64

In [9]:
test.isnull().sum(axis = 0)

ticket_id          0
violation_code     0
disposition        0
fine_amount        0
late_fee           0
discount_amount    0
clean_up_cost      0
judgment_amount    0
lat                0
lon                0
dtype: int64

In [10]:
# Testing Random Forest Classifier
X_train, X_test, y_train, y_test = train_test_split(train[train_columns], train['compliance'])
    
regr_rf = RandomForestRegressor()
grid_values = {'n_estimators': [10, 100], 'max_depth': [None, 30]}
grid_clf_auc = GridSearchCV(regr_rf, param_grid=grid_values, scoring='roc_auc')
grid_clf_auc.fit(X_train, y_train)
print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
print('Grid best score (AUC): ', grid_clf_auc.best_score_)

result = pd.DataFrame(grid_clf_auc.predict(test), test.ticket_id)

Grid best parameter (max. AUC):  {'n_estimators': 100, 'max_depth': 30}
Grid best score (AUC):  0.812428464684
